# Training Notebook

Library Imports for the jupyter notebook

In [1]:
import os   # miscellaneous os interfaces
import sys  # configuring python runtime environment
import time # library for time manipulation, and logging

In [2]:
# use `datetime` to control and preceive the environment
# in addition `pandas` also provides date time functionalities
import datetime as dt

In [3]:
from copy import deepcopy      # dataframe is mutable
from tqdm import tqdm     # progress bar for loops
from uuid import uuid4 as UUID # unique identifier for objs

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
!pip install torchsummary

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms

from torchinfo import summary
from torch.utils.data import DataLoader
from torchsummary import summary